# Clase 1

In [ ]:
import findspark
import pyspark

In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType


In [3]:
schema=StructType().add("user_id","string").add("country","string").add("browser", "string").add("OS",'string').add("age", "integer")


In [4]:
df=spark.createDataFrame([("A203",'India',"Chrome","WIN", 33),("A201",'China',"Safari","MacOS",35),("A205",'UK',"Mozilla", "Linux",25)],schema=schema)


In [5]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- age: integer (nullable = true)



In [6]:
df.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|  India| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [7]:
df_na=spark.createDataFrame([("A203",None,"Chrome","WIN", 33),("A201",'China',None,"MacOS",35),("A205",'UK',"Mozilla", "Linux",25)],schema=schema)


In [8]:
df_na.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|   null| Chrome|  WIN| 33|
|   A201|  China|   null|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [9]:
df_na.fillna('0').show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|      0| Chrome|  WIN| 33|
|   A201|  China|      0|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [10]:
df_na.fillna({'country':'Usa','browser':'Safari'}).show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|    Usa| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [11]:
df_na.na.drop(subset=['browser']).show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|   null| Chrome|  WIN| 33|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [12]:
df_na.replace('Chrome','Google Chrome').show()

+-------+-------+-------------+-----+---+
|user_id|country|      browser|   OS|age|
+-------+-------+-------------+-----+---+
|   A203|   null|Google Chrome|  WIN| 33|
|   A201|  China|         null|MacOS| 35|
|   A205|     UK|      Mozilla|Linux| 25|
+-------+-------+-------------+-----+---+



In [13]:
df = spark.read.csv('customer_data.csv',header = True, inferSchema = True)

In [14]:
df.count()

2000

In [15]:
df.columns

['Customer_subtype',
 'Number_of_houses',
 'Avg_size_household',
 'Avg_age',
 'Customer_main_type',
 'Avg_Salary',
 'label']

In [16]:
df.show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0|
|Modern, complete ...|               1|                 3|40-50 years|      Average Family|     19504|    0|
|  Large family farms|               1|                 4|30-40 years|             Farmers|     34943|    0|
|    Young and rising|               1|                 2|20-30 years|         Living well|     13064|    0|
|Large religious f.

In [17]:
df.printSchema()

root
 |-- Customer_subtype: string (nullable = true)
 |-- Number_of_houses: integer (nullable = true)
 |-- Avg_size_household: integer (nullable = true)
 |-- Avg_age: string (nullable = true)
 |-- Customer_main_type: string (nullable = true)
 |-- Avg_Salary: integer (nullable = true)
 |-- label: integer (nullable = true)



In [18]:
df.summary().show()

+-------+--------------------+------------------+------------------+-----------+--------------------+-----------------+------------------+
|summary|    Customer_subtype|  Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|       Avg_Salary|             label|
+-------+--------------------+------------------+------------------+-----------+--------------------+-----------------+------------------+
|  count|                2000|              2000|              2000|       2000|                2000|             2000|              2000|
|   mean|                null|            1.1075|            2.6895|       null|                null|     1616908.0835|            0.0605|
| stddev|                null|0.3873225521186316|0.7914562220841646|       null|                null|6822647.757312146|0.2384705099001677|
|    min|Affluent senior a...|                 1|                 1|20-30 years|      Average Family|             1361|                 0|
|    25%|                nu

In [19]:
df.select(['Customer_subtype','Avg_Salary']).show(5)

+--------------------+----------+
|    Customer_subtype|Avg_Salary|
+--------------------+----------+
|Lower class large...|     44905|
|Mixed small town ...|     37575|
|Mixed small town ...|     27915|
|Modern, complete ...|     19504|
|  Large family farms|     34943|
+--------------------+----------+
only showing top 5 rows



In [20]:
df.filter(df['Avg_Salary']>1000000).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
| High status seniors|               1|                 3|40-50 years|Successful hedonists|   4670288|    0|
| High status seniors|               1|                 3|50-60 years|Successful hedonists|   9561873|    0|
| High status seniors|               1|                 2|40-50 years|Successful hedonists|  18687005|    0|
| High status seniors|               1|                 2|40-50 years|Successful hedonists|  24139960|    0|
| High status seniors|               1|                 2|50-60 years|Successful hedonists|   6718606|    0|
|High Income, expe...|               1|                 3|40-50 years|Successful hedonists|  19347139|    0|
|High Income, expe.

In [21]:
df.filter(df['Avg_Salary']>1000000).count()

128

In [24]:
df.filter(df['Avg_Salary']>500000).filter(df['Number_of_houses']>2).count()

4

In [25]:
df.filter(df['Avg_Salary']>500000).filter(df['Number_of_houses']>2).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    596723|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    788477|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+



In [26]:
df.where((df['Avg_Salary']>500000) & (df['Number_of_houses']>2)).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    596723|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    788477|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+



In [27]:
df.groupby('Number_of_houses').count().show()

+----------------+-----+
|Number_of_houses|count|
+----------------+-----+
|               1| 1808|
|               3|   12|
|               5|    1|
|              10|    1|
|               2|  178|
+----------------+-----+



In [28]:
df.groupby('Customer_subtype').agg(F.mean('Avg_Salary')).show()

+--------------------+--------------------+
|    Customer_subtype|     avg(Avg_Salary)|
+--------------------+--------------------+
|Large family, emp...|  32867.857142857145|
|Religious elderly...|   30540.59574468085|
|Large religious f...|  29652.196261682242|
|Modern, complete ...|             31576.0|
|    Village families|  32449.470588235294|
|Young all america...|    29563.3064516129|
|Young urban have-...|             25751.0|
|Young seniors in ...|  30105.136363636364|
|Fresh masters in ...|             27645.0|
|High Income, expe...|2.3839817807692308E7|
|Lower class large...|  26012.628472222223|
| Residential elderly|  28866.166666666668|
|Senior cosmopolitans|             49903.0|
|        Mixed rurals|  29073.761194029852|
|Career and childcare|  30110.939393939392|
|Low income catholics|  21713.777777777777|
|Mixed apartment d...|  29587.647058823528|
|Seniors in apartm...|  30090.882352941175|
|Middle class fami...|  31579.385245901638|
|Traditional families|   29381.8